In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column,Integer,String,Numeric,Text,Float,ForeignKey
from sqlalchemy.orm import sessionmaker,relationship
import datetime
import dateutil.relativedelta
import quandl

In [2]:
engine = create_engine("sqlite:///pub_atx.sqlite")

In [3]:
Base = declarative_base()

In [4]:
class Company(Base):
    __tablename__ = 'cmpny'
    
    id_cmpny = Column(Integer, primary_key=True)
    rank = Column(Integer)
    name = Column(Text)
    address = Column(Text)
    city = Column(Text)
    state = Column(Text)
    zip_code = Column(Text)
    phn_nbr = Column(Text)
    website = Column(Text)
    tckr = Column(Text)
    ipo_yr = Column(Integer)
    yr_founded = Column(Integer)
    gen_buss_desc = Column(Text)
    curr_top_exec = Column(Text)
    yr_estblsh = Column(Integer)
    austin_staff_cnt = Column(Float)
    comp_staff_cnt = Column(Float)
    tot_local_emp_cnt = Column(Float)
    exchng = Column(Text)
    lat = Column(Float)
    lng = Column(Float)
    
    def __repr__(self):
        return f"id={self.id},name=[self.name]"
    
class CompanyPrcsMnthly(Base):
    __tablename__ = 'cmpny_prcs_mnthly'
    
    id_cmpny_prcs_mnthly = Column(Integer, primary_key=True)
    id_cmpny = Column(Integer)
    date = Column(Text)
    open = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    volume = Column(Float)
    
    
    def __repr__(self):
        return f"id={self.id_cmpny_prcs_mnthly},name=[self.id_cmpny]"
    
class CompanyPrcsDaily(Base):
    __tablename__ = 'cmpny_prcs_daily'
    
    id_cmpny_prcs_daily = Column(Integer, primary_key=True)
    id_cmpny = Column(Integer)
    date = Column(Text)
    open = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    volume = Column(Float)
    
    
    def __repr__(self):
        return f"id={self.id_cmpny_prcs_daily},name=[self.id_cmpny]"

In [5]:
!rm pub_atx.sqlite

rm: pub_atx.sqlite: No such file or directory


In [6]:
Base.metadata.create_all(engine)

In [7]:
engine.table_names()

['cmpny', 'cmpny_prcs_daily', 'cmpny_prcs_mnthly']

In [8]:
conn =engine.connect()
df_company = pd.read_csv("clean_company_list.csv")
data = df_company.to_dict(orient='records')
conn.execute(Company.__table__.delete())
conn.execute(Company.__table__.insert(),data)

In [9]:
companies = engine.execute("select * from cmpny").fetchall()

In [10]:
companies

[(1, 1, 'Whole Foods Market Inc.', '550 Bowie St.', 'Austin', 'TX', '78703', '512-476-1206', 'wholefoodsmarket.com', 'WFMI', 1992, 1980, 'Retail grocery of natural and organic foods; operates 431 stores:U.S. (412), Canada (10), U.K.(9)', 'John  Mackey,  CEO', 1980, 3000.0, 87000.0, 2530.0, 'Nasdaq', 30.212466999999997, -97.7439969),
 (2, 2, 'National Instruments Corp.', '11500 N. MoPac Expy.', 'Austin', 'TX', '78759', '512-683-0100', 'ni.com', 'NATI', 1995, 1976, 'Designs, manufactures and sells tools to engineers and scientists, equipping them with systems that accelerate productivity, innovation and discovery ', 'Alex  Davern,  President/CEO', 1976, 2500.0, 7552.0, 2600.0, 'Nasdaq', 30.212466999999997, -97.7439969),
 (3, 3, 'Cirrus Logic Inc.', '800 W. Sixth St.', 'Austin', 'TX', '78701', '512-851-4000', 'cirrus.com', 'CRUS', 1989, 1984, 'Semiconductor developing high-precision, analog and mixed-signal integrated circuits for audio and energy applications', 'Jason  Rhode,  President/

In [12]:

d1 = datetime.date.today()
d2 = d1 - dateutil.relativedelta.relativedelta(months=36)
start_dt = d2.strftime("%Y-%m-%d")
end_dt = d1.strftime("%Y-%m-%d")


In [13]:
quandl.ApiConfig.api_key = ""


In [14]:
conn.execute(CompanyPrcsMnthly.__table__.delete())

In [15]:
for company in companies:
    try:
       
        data = quandl.get("WIKI" +"/" + company.tckr,collapse="monthly", start_date=start_dt, end_date=end_dt)
        data = data.reset_index()
        data = data[['Date','Open','High','Low','Close','Volume']]
        data.columns = ["date","open","high","low","close","volume"]
        data["id_cmpny"] = company.id_cmpny
        data = data[["id_cmpny", "date","open","high","low","close","volume"]]
        data['date'] = data['date'].astype('str')
        data = data.to_dict(orient='records')
        
        conn.execute(CompanyPrcsMnthly.__table__.insert(),data)
    except quandl.NotFoundError as e:
        print(company.tckr + " not found ")
        continue

WFMI not found 
BETR not found 
USAC not found 
XPLR not found 
APPS not found 
STRS not found 
UPLD not found 
ASUR not found 
AGLE not found 
FPP not found 
ASTC not found 
VRML not found 
IPWR not found 
CRDS not found 
SVRA not found 
VYEY not found 
SCON not found 
XBIT not found 
MIRN not found 


In [16]:
conn.execute(CompanyPrcsDaily.__table__.delete())

In [17]:
for company in companies:
    try:
       
        data = quandl.get("WIKI" +"/" + company.tckr, start_date=start_dt, end_date=end_dt)
        data = data.reset_index()
        data = data[['Date','Open','High','Low','Close','Volume']]
        data.columns = ["date","open","high","low","close","volume"]
        data["id_cmpny"] = company.id_cmpny
        data = data[["id_cmpny", "date","open","high","low","close","volume"]]
        data['date'] = data['date'].astype('str')
        data = data.to_dict(orient='records')
        
        conn.execute(CompanyPrcsDaily.__table__.insert(),data)
    except quandl.NotFoundError as e:
        print(company.tckr + " not found ")
        continue

WFMI not found 
BETR not found 
USAC not found 
XPLR not found 
APPS not found 
STRS not found 
UPLD not found 
ASUR not found 
AGLE not found 
FPP not found 
ASTC not found 
VRML not found 
IPWR not found 
CRDS not found 
SVRA not found 
VYEY not found 
SCON not found 
XBIT not found 
MIRN not found 


In [18]:

engine.execute("select count(*) from cmpny_prcs_mnthly").fetchall()

[(591,)]

In [19]:
engine.execute("select count(*) from cmpny_prcs_daily").fetchall()

[(12063,)]

In [20]:

engine.execute("select * from cmpny_prcs_mnthly").fetchall()

[(1, 2, '2015-02-28', 31.13, 31.28, 31.0, 31.14, 267234.0),
 (2, 2, '2015-03-31', 31.94, 32.14, 31.84, 32.04, 381739.0),
 (3, 2, '2015-04-30', 28.97, 29.99, 28.27, 28.6, 1049694.0),
 (4, 2, '2015-05-31', 30.18, 30.24, 29.7602, 29.91, 430723.0),
 (5, 2, '2015-06-30', 29.83, 29.83, 29.33, 29.46, 384339.0),
 (6, 2, '2015-07-31', 28.01, 29.34, 27.94, 28.96, 840170.0),
 (7, 2, '2015-08-31', 29.05, 29.3, 28.83, 29.21, 507588.0),
 (8, 2, '2015-09-30', 27.43, 27.93, 27.28, 27.79, 302592.0),
 (9, 2, '2015-10-31', 30.78, 31.1, 30.45, 30.47, 334999.0),
 (10, 2, '2015-11-30', 31.2, 31.7, 31.0, 31.4, 307919.0),
 (11, 2, '2015-12-31', 28.99, 29.625, 28.67, 28.69, 199485.0),
 (12, 2, '2016-01-31', 28.63, 29.22, 28.43, 28.5, 633896.0),
 (13, 2, '2016-02-29', 28.84, 29.18, 28.5301, 28.85, 253374.0),
 (14, 2, '2016-03-31', 29.88, 30.42, 29.88, 30.11, 317102.0),
 (15, 2, '2016-04-30', 27.75, 28.345, 27.32, 27.57, 432564.0),
 (16, 2, '2016-05-31', 28.52, 28.71, 28.38, 28.57, 645419.0),
 (17, 2, '2016-06-3

In [21]:
engine.execute("select * from cmpny_prcs_daily").fetchall()

[(1, 2, '2015-02-25', 31.15, 31.4, 31.01, 31.32, 362391.0),
 (2, 2, '2015-02-26', 31.25, 31.46, 31.1, 31.18, 389642.0),
 (3, 2, '2015-02-27', 31.13, 31.28, 31.0, 31.14, 267234.0),
 (4, 2, '2015-03-02', 31.1, 31.61, 31.1, 31.55, 513666.0),
 (5, 2, '2015-03-03', 31.54, 31.55, 31.07, 31.13, 300668.0),
 (6, 2, '2015-03-04', 31.08, 31.26, 30.87, 31.18, 239188.0),
 (7, 2, '2015-03-05', 31.26, 31.5, 31.1, 31.38, 199828.0),
 (8, 2, '2015-03-06', 31.17, 31.38, 30.94, 31.09, 249147.0),
 (9, 2, '2015-03-09', 31.07, 31.22, 30.97, 31.18, 202587.0),
 (10, 2, '2015-03-10', 30.89, 31.26, 30.51, 30.6, 257958.0),
 (11, 2, '2015-03-11', 30.6, 31.27, 30.52, 31.27, 363270.0),
 (12, 2, '2015-03-12', 31.4, 31.45, 30.86, 31.34, 474558.0),
 (13, 2, '2015-03-13', 31.31, 31.48, 31.1, 31.38, 363479.0),
 (14, 2, '2015-03-16', 31.47, 32.15, 31.35, 32.07, 408193.0),
 (15, 2, '2015-03-17', 32.11, 32.56, 31.96, 32.44, 495798.0),
 (16, 2, '2015-03-18', 32.5, 32.96, 32.25, 32.85, 459091.0),
 (17, 2, '2015-03-19', 32.81,

[{'close': 1.9,
  'date': '2015-02-28',
  'high': 1.98,
  'id_cmpny': 34,
  'low': 1.9,
  'open': 1.9,
  'volume': 100149.0},
 {'close': 1.89,
  'date': '2015-03-31',
  'high': 1.9499,
  'id_cmpny': 34,
  'low': 1.87,
  'open': 1.88,
  'volume': 60519.0},
 {'close': 2.04,
  'date': '2015-04-30',
  'high': 2.1,
  'id_cmpny': 34,
  'low': 2.04,
  'open': 2.09,
  'volume': 137974.0},
 {'close': 1.97,
  'date': '2015-05-31',
  'high': 2.07,
  'id_cmpny': 34,
  'low': 1.95,
  'open': 2.02,
  'volume': 418525.0},
 {'close': 1.73,
  'date': '2015-06-30',
  'high': 1.74,
  'id_cmpny': 34,
  'low': 1.6,
  'open': 1.65,
  'volume': 413176.0},
 {'close': 1.89,
  'date': '2015-07-31',
  'high': 1.92,
  'id_cmpny': 34,
  'low': 1.74,
  'open': 1.74,
  'volume': 238894.0},
 {'close': 1.84,
  'date': '2015-08-31',
  'high': 1.93,
  'id_cmpny': 34,
  'low': 1.8,
  'open': 1.82,
  'volume': 95304.0},
 {'close': 1.83,
  'date': '2015-09-30',
  'high': 1.83,
  'id_cmpny': 34,
  'low': 1.73,
  'open': 1.8